Script to analyse data related to fisheries. These data concerns:

Target 14.4 (FMSY/F and B/BMSY)

Target 14.6 (TAC/Catch)

Target 14.a (SAD/TAC) 

1. Difference between officialNominalCatces and catches from StockAssesment dataset. For what stock do we have large differences? <10% would be alright. 
2. Stock Assesment is not done every year. Rudi: Do they provide a trend for those years that they don't assess? It's odd to use different years for different stocks, but it's possible. 
3. BMSY in ICES. Btrigger Bpa is the same as Btrigger, go for that one. 
4. Effort is used because it's hard to track the catches. Send questions to Rudi


In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
os.chdir('..')
os.getcwd()

'c:\\Users\\isaldiviagonzatti\\OneDrive - Kiel Institut für Weltwirtschaft\\Documents\\General\\Ignacio'

In [4]:
countries=['Belgium','Bulgaria','Cyprus', 'Greece','Germany','Croatia','Italy', 
           'Denmark','Estonia','Spain','Finland','France','Ireland','Lithuania',
           'Latvia','Malta','Netherlands','Poland','Portugal', 'Romania',
           'Sweden','United Kingdom of Great Britain and Northern Ireland', "United Kingdom of GB"]

In [5]:
country_to_abbrev = {
    "Andorra": "AD",
    "United Arab Emirates": "AE",
    "Afghanistan": "AF",
    "Antigua and Barbuda": "AG",
    "Anguilla": "AI",
    "Albania": "AL",
    "Armenia": "AM",
    "Angola": "AO",
    "Antarctica": "AQ",
    "Argentina": "AR",
    "American Samoa": "AS",
    "Austria": "AT",
    "Australia": "AU",
    "Aruba": "AW",
    "Åland Islands": "AX",
    "Azerbaijan": "AZ",
    "Bosnia and Herzegovina": "BA",
    "Barbados": "BB",
    "Bangladesh": "BD",
    "Belgium": "BE",
    "Burkina Faso": "BF",
    "Bulgaria": "BG",
    "Bahrain": "BH",
    "Burundi": "BI",
    "Benin": "BJ",
    "Saint Barthélemy": "BL",
    "Bermuda": "BM",
    "Brunei Darussalam": "BN",
    "Bolivia (Plurinational State of)": "BO",
    "Bonaire, Sint Eustatius and Saba": "BQ",
    "Brazil": "BR",
    "Bahamas": "BS",
    "Bhutan": "BT",
    "Bouvet Island": "BV",
    "Botswana": "BW",
    "Belarus": "BY",
    "Belize": "BZ",
    "Canada": "CA",
    "Cocos (Keeling) Islands": "CC",
    "Congo, Democratic Republic of the": "CD",
    "Central African Republic": "CF",
    "Congo": "CG",
    "Switzerland": "CH",
    "Côte d'Ivoire": "CI",
    "Cook Islands": "CK",
    "Chile": "CL",
    "Cameroon": "CM",
    "China": "CN",
    "Colombia": "CO",
    "Costa Rica": "CR",
    "Cuba": "CU",
    "Cabo Verde": "CV",
    "Curaçao": "CW",
    "Christmas Island": "CX",
    "Cyprus": "CY",
    "Czechia": "CZ",
    "Germany": "DE",
    "Djibouti": "DJ",
    "Denmark": "DK",
    "Dominica": "DM",
    "Dominican Republic": "DO",
    "Algeria": "DZ",
    "Ecuador": "EC",
    "Estonia": "EE",
    "Egypt": "EG",
    "Western Sahara": "EH",
    "Eritrea": "ER",
    "Spain": "ES",
    "Ethiopia": "ET",
    "Finland": "FI",
    "Fiji": "FJ",
    "Falkland Islands (Malvinas)": "FK",
    "Micronesia (Federated States of)": "FM",
    "Faroe Islands": "FO",
    "France": "FR",
    "Gabon": "GA",
    "United Kingdom of Great Britain and Northern Ireland": "UK", #original is GB, Eurostat uses UK
    "United Kingdom of GB": "GB",
    "Grenada": "GD",
    "Georgia": "GE",
    "French Guiana": "GF",
    "Guernsey": "GG",
    "Ghana": "GH",
    "Gibraltar": "GI",
    "Greenland": "GL",
    "Gambia": "GM",
    "Guinea": "GN",
    "Guadeloupe": "GP",
    "Equatorial Guinea": "GQ",
    "Greece": "EL", #original ir GR, Eurostat uses EL
    "South Georgia and the South Sandwich Islands": "GS",
    "Guatemala": "GT",
    "Guam": "GU",
    "Guinea-Bissau": "GW",
    "Guyana": "GY",
    "Hong Kong": "HK",
    "Heard Island and McDonald Islands": "HM",
    "Honduras": "HN",
    "Croatia": "HR",
    "Haiti": "HT",
    "Hungary": "HU",
    "Indonesia": "ID",
    "Ireland": "IE",
    "Israel": "IL",
    "Isle of Man": "IM",
    "India": "IN",
    "British Indian Ocean Territory": "IO",
    "Iraq": "IQ",
    "Iran (Islamic Republic of)": "IR",
    "Iceland": "IS",
    "Italy": "IT",
    "Jersey": "JE",
    "Jamaica": "JM",
    "Jordan": "JO",
    "Japan": "JP",
    "Kenya": "KE",
    "Kyrgyzstan": "KG",
    "Cambodia": "KH",
    "Kiribati": "KI",
    "Comoros": "KM",
    "Saint Kitts and Nevis": "KN",
    "Korea (Democratic People's Republic of)": "KP",
    "Korea, Republic of": "KR",
    "Kuwait": "KW",
    "Cayman Islands": "KY",
    "Kazakhstan": "KZ",
    "Lao People's Democratic Republic": "LA",
    "Lebanon": "LB",
    "Saint Lucia": "LC",
    "Liechtenstein": "LI",
    "Sri Lanka": "LK",
    "Liberia": "LR",
    "Lesotho": "LS",
    "Lithuania": "LT",
    "Luxembourg": "LU",
    "Latvia": "LV",
    "Libya": "LY",
    "Morocco": "MA",
    "Monaco": "MC",
    "Moldova, Republic of": "MD",
    "Montenegro": "ME",
    "Saint Martin (French part)": "MF",
    "Madagascar": "MG",
    "Marshall Islands": "MH",
    "North Macedonia": "MK",
    "Mali": "ML",
    "Myanmar": "MM",
    "Mongolia": "MN",
    "Macao": "MO",
    "Northern Mariana Islands": "MP",
    "Martinique": "MQ",
    "Mauritania": "MR",
    "Montserrat": "MS",
    "Malta": "MT",
    "Mauritius": "MU",
    "Maldives": "MV",
    "Malawi": "MW",
    "Mexico": "MX",
    "Malaysia": "MY",
    "Mozambique": "MZ",
    "Namibia": "NA",
    "New Caledonia": "NC",
    "Niger": "NE",
    "Norfolk Island": "NF",
    "Nigeria": "NG",
    "Nicaragua": "NI",
    "Netherlands": "NL",
    "Norway": "NO",
    "Nepal": "NP",
    "Nauru": "NR",
    "Niue": "NU",
    "New Zealand": "NZ",
    "Oman": "OM",
    "Panama": "PA",
    "Peru": "PE",
    "French Polynesia": "PF",
    "Papua New Guinea": "PG",
    "Philippines": "PH",
    "Pakistan": "PK",
    "Poland": "PL",
    "Saint Pierre and Miquelon": "PM",
    "Pitcairn": "PN",
    "Puerto Rico": "PR",
    "Palestine, State of": "PS",
    "Portugal": "PT",
    "Palau": "PW",
    "Paraguay": "PY",
    "Qatar": "QA",
    "Réunion": "RE",
    "Romania": "RO",
    "Serbia": "RS",
    "Russian Federation": "RU",
    "Rwanda": "RW",
    "Saudi Arabia": "SA",
    "Solomon Islands": "SB",
    "Seychelles": "SC",
    "Sudan": "SD",
    "Sweden": "SE",
    "Singapore": "SG",
    "Saint Helena, Ascension and Tristan da Cunha": "SH",
    "Slovenia": "SI",
    "Svalbard and Jan Mayen": "SJ",
    "Slovakia": "SK",
    "Sierra Leone": "SL",
    "San Marino": "SM",
    "Senegal": "SN",
    "Somalia": "SO",
    "Suriname": "SR",
    "South Sudan": "SS",
    "Sao Tome and Principe": "ST",
    "El Salvador": "SV",
    "Sint Maarten (Dutch part)": "SX",
    "Syrian Arab Republic": "SY",
    "Eswatini": "SZ",
    "Turks and Caicos Islands": "TC",
    "Chad": "TD",
    "French Southern Territories": "TF",
    "Togo": "TG",
    "Thailand": "TH",
    "Tajikistan": "TJ",
    "Tokelau": "TK",
    "Timor-Leste": "TL",
    "Turkmenistan": "TM",
    "Tunisia": "TN",
    "Tonga": "TO",
    "Turkey": "TR",
    "Trinidad and Tobago": "TT",
    "Tuvalu": "TV",
    "Taiwan, Province of China": "TW",
    "Tanzania, United Republic of": "TZ",
    "Ukraine": "UA",
    "Uganda": "UG",
    "United States Minor Outlying Islands": "UM",
    "United States of America": "US",
    "Uruguay": "UY",
    "Uzbekistan": "UZ",
    "Holy See": "VA",
    "Saint Vincent and the Grenadines": "VC",
    "Venezuela (Bolivarian Republic of)": "VE",
    "Virgin Islands (British)": "VG",
    "Virgin Islands (U.S.)": "VI",
    "Viet Nam": "VN",
    "Vanuatu": "VU",
    "Wallis and Futuna": "WF",
    "Samoa": "WS",
    "Yemen": "YE",
    "Mayotte": "YT",
    "South Africa": "ZA",
    "Zambia": "ZM",
    "Zimbabwe": "ZW",
}
    
# invert the dictionary
abbrev_to_country = dict(map(reversed, country_to_abbrev.items()))

## Data manipulation

### icesTACcomparison

In [6]:
pd.ExcelFile(os.path.realpath("data/icesTACcomparison.xlsx")).sheet_names

['Menus',
 'Table of contents',
 'ICES advice',
 'Council agreed TAC',
 'Comparison',
 'Table for results',
 'Overall results',
 'Results by Member State',
 'Sea basin',
 'Third country',
 'Results by % difference',
 'Results by # of TACs',
 'Results by # of TACs by MS',
 'Results by species',
 'ID',
 'Matching - ICES-TAC',
 'Matching - Final TACs',
 'Matching - EU share',
 'Matching - TAC split share',
 'Matching - ICES area share']

In [7]:
pd.read_excel(os.path.realpath("data/icesTACcomparison.xlsx"), 1)

,Tab,Description
0,ICES advice,ICES advice by TAC and year
1,Council agreed TAC,Council agreed TAC by Member State and year
2,Comparison,Comparing ICES advice and agreed TACs by Membe...
3,Overall results,Calculates the difference between TACs and ICE...
4,Results by Member State,Calculates the difference between TACs and ICE...
5,Results by % difference,Calculates the difference between TACs and ICE...
6,Results by # of TACs,Calculates the number of TACs that exceed ICES...
7,Results by # of TACs by MS,Calculates the number of TACs that exceed ICES...
8,Results by third country share,Calculates the difference between TACs and ICE...
9,Results by species,Calculates the difference between TACs and ICE...


#### TAC

In [8]:
tac = pd.read_excel(os.path.realpath("data/icesTACcomparison.xlsx"), 'Council agreed TAC')

In [9]:
# extract acronym of species and convert to lower case
tac["speciesAcronym"] = tac["Reference"].str.extract(r"\(([\w\-]+)" , expand=False).str.lower()

In [11]:
tac[(tac.Species == 'Plaice') & (tac.Year.isin([2012,2016,2020])) & (tac.Level == 'TAC') & (tac['ICES area'] == 4)  ]

KeyError: 'ICES area'

#### ICES advice

In [12]:
sad = pd.read_excel(os.path.realpath("data/icesTACcomparison.xlsx"), 'ICES advice') 

In [13]:
sad[(sad['ICES code'] == 'ple.27.420') & (sad.Year.isin([2012,2016,2020]))]

,Code,Species,ICES code,ICES area,Advice,Year,Sea,Choices,ICES advice,Catches corresponding to advice,Landings corresponding to advice,Unnamed: 11
1507,ple.27.420 - 2012,Plaice,ple.27.420,4,84410,2012,Atlantic,NaN,Apply first stage of the management plan,NaN,84410,NaN
1511,ple.27.420 - 2016,Plaice,ple.27.420,4,152910,2016,Atlantic,North sea split out,MSY approach,216345,152910,NaN
1515,ple.27.420 - 2020,Plaice,ple.27.420,4,166499,2020,Atlantic,North sea split out,MSY approach,166499,NaN,NaN


#### TAC-SAC Comparison

In [14]:
sad_tac = pd.read_excel(os.path.realpath("data/icesTACcomparison.xlsx"), 'Comparison') 

In [15]:
sad_tac[(sad_tac.Species == 'Plaice') & (sad_tac.Year.isin([2012,2016,2020])) & (sad_tac.Level == 'TAC') & (sad_tac['ICES area'] == 4)  ]

,Final Ref,Reference,TAC ID,Species,TAC Zone,ICES area,Level,TAC,Year,Amendment/Original,Sea,Amendment check,Agreement,ICES Advice,TAC above advice (t),TAC above advice (%),Net difference (t),Net difference (%),Advice change ICES,Advice change %,Advice change quantity,Previous year TAC,ICES with Council
16399,(PLE/03AN.) - 2016 - TAC - Final,(PLE/03AN.) - 2016 - TAC - Original,(PLE/03AN.),Plaice,Skagerrak,4,TAC,10056.410256,2016,Original,Atlantic,Final,Norway joint management,11108.11597,0.0,0.0,-1051.705714,-0.094679,0.766839,4.512713,-9093.11597,2015.0,11108.11597
16431,(PLE/03AN.) - 2020 - TAC - Final,(PLE/03AN.) - 2020 - TAC - Original,(PLE/03AN.),Plaice,Skagerrak,4,TAC,19647,2020,Original,Atlantic,Final,Norway joint management,19647,0.0,0.0,0.000000,0,0.170719,8.731055,-17628.00000,2019.0,19647
16808,(PLE/2A3AX4) - 2012 - TAC - Final,(PLE/2A3AX4) - 2012 - TAC - Original,(PLE/2A3AX4),Plaice,4; Union waters of 2a; that part of 3a not cov...,4,TAC,84410,2012,Original,Atlantic,Final,Norway joint management,84410,0.0,0.0,0.000000,0,0.314798,40.974142,-82399.00000,2011.0,84410
16862,(PLE/2A3AX4) - 2016 - TAC - Final,(PLE/2A3AX4) - 2016 - TAC - Original,(PLE/2A3AX4),Plaice,4; Union waters of 2a; that part of 3a not cov...,4,TAC,128376.218324,2016,Original,Atlantic,Final,Norway joint management,141801.88403,0.0,0.0,-13425.665706,-0.094679,0.104583,69.373143,-139786.88403,2015.0,141801.88403
16898,(PLE/2A3AX4) - 2020 - TAC - Final,(PLE/2A3AX4) - 2020 - TAC - Original,(PLE/2A3AX4),Plaice,4; Union waters of 2a; that part of 3a not cov...,4,TAC,146852,2020,Original,Atlantic,Final,Norway joint management,146852,0.0,0.0,0.000000,0,0.170742,71.735017,-144833.00000,2019.0,146852


### StockAssessment

In [16]:
stock = pd.read_csv(os.path.relpath('data/stockAssesment/StockAssessment.csv'), names=range(138))
stock.columns = stock.iloc[0,:]
stock = stock[1:]

C:\Users\isaldiviagonzatti\AppData\Local\Temp\ipykernel_24928\1294945985.py:1: DtypeWarning: Columns (0,1,3,4,6,10,12,13,14,17,18,19,20,21,22,23,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,46,47,48,49,50,51,52,54,56,57,58,59,60,62,65,66,68,69,70,71,72,73,74,75,76,79,82,85,88,91,94,97,100,103,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,137) have mixed types. Specify dtype option on import or set low_memory=False.
  stock = pd.read_csv(os.path.relpath('data/stockAssesment/StockAssessment.csv'), names=range(138))


In [17]:
# extract EN name and acronym of species
stock['enName'] = stock["StockDescription"].str.extract(r"^(.+?) ?(?:\d|\(|$)" , expand=False)
stock['speciesAcronym'] = stock["FishStock"].str.extract(r"^([^.]*).*" , expand=False)

In [18]:
# print(stock.columns.tolist())

In [19]:
# select useful years and columns
stock = stock.loc[:,['Year','enName', 'speciesAcronym','FishStock','StockKey', 'SpeciesName', "ICES Areas (splited with character '~')", 
                     'StockSize', 'StockSizeDescription', 'StockSizeUnits',
                      'FishingPressure', 'FishingPressureDescription', 'FishingPressureUnits',
                     'Flim', 'Fpa', 'Blim', 'Bpa', 'FMSY', 'MSYBtrigger', 
                     'CatchesLadingsUnits', 'Landings', 'OfficialLandings', 'Catches',
                     'Report']]
stock.Year = stock.Year.astype(int)
stock = stock[stock.Year.isin([2012,2016,2020])]

In [20]:
# To merge with OfficialCatches database
# Split the areas column into several columns and melt to get one row per species-area (as per in the CatchesOfficial) 

sepAreas = stock["ICES Areas (splited with character '~')"].str.split('~', expand=True)
sepAreas = sepAreas.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

#melt by leaving out all the columns containing the areas from "ICES Areas (splited with character '~')"
stockMelt = stock.join(sepAreas)
stockMelt = stockMelt.melt(id_vars=['Year', 'enName', 'speciesAcronym', 'FishStock', 'StockKey',
       'SpeciesName', "ICES Areas (splited with character '~')", 'StockSize',
       'StockSizeDescription', 'StockSizeUnits', 'FishingPressure',
       'FishingPressureDescription', 'FishingPressureUnits', 'Flim', 'Fpa',
       'Blim', 'Bpa', 'FMSY', 'MSYBtrigger', 'CatchesLadingsUnits', 'Landings',
       'OfficialLandings', 'Catches', 'Report'], 
       var_name='fullArea', value_name='area')
stockMelt = stockMelt.dropna(subset=['area'])


# (later realised could have done it with explode)

stockExplode = (stock.set_index(['Year', 'enName', 'speciesAcronym', 'FishStock', 'StockKey',
       'SpeciesName', 'StockSize',
       'StockSizeDescription', 'StockSizeUnits', 'FishingPressure',
       'FishingPressureDescription', 'FishingPressureUnits', 'Flim', 'Fpa',
       'Blim', 'Bpa', 'FMSY', 'MSYBtrigger', 'CatchesLadingsUnits', 'Landings',
       'OfficialLandings', 'Catches', 'Report'])
   .apply(lambda x: x.str.split('~').explode())
   .reset_index())  
stockExplode["ICES Areas (splited with character '~')"] = stockExplode["ICES Areas (splited with character '~')"].str.strip()

In [21]:
# there are duplicates because of updated reports, we drop them keeping the last version
stockMelt = stockMelt.drop_duplicates(subset=['Year', 'area', 'FishStock'], keep='last')

In [22]:
# check result by species
stockMelt[(stockMelt.FishStock== 'san.sa.1r') & (stockMelt["ICES Areas (splited with character '~')"] == '27.4.b ~ 27.4.c') ]

,Year,enName,speciesAcronym,FishStock,StockKey,SpeciesName,ICES Areas (splited with character '~'),StockSize,StockSizeDescription,StockSizeUnits,FishingPressure,FishingPressureDescription,FishingPressureUnits,Flim,Fpa,Blim,Bpa,FMSY,MSYBtrigger,CatchesLadingsUnits,Landings,OfficialLandings,Catches,Report,fullArea,area
0,2012,Sandeel,san,san.sa.1r,169246,Ammodytes,27.4.b ~ 27.4.c,152970,SSB,tonnes,0.112,F,Year-1,NaN,NaN,110000,145000,NaN,NaN,tonnes,NaN,NaN,45954,https://doi.org/10.17895/ices.advice.10000,0,27.4.b
1,2016,Sandeel,san,san.sa.1r,169246,Ammodytes,27.4.b ~ 27.4.c,231422,SSB,tonnes,0.028,F,Year-1,NaN,NaN,110000,145000,NaN,NaN,tonnes,NaN,NaN,15407,https://doi.org/10.17895/ices.advice.10000,0,27.4.b
2,2020,Sandeel,san,san.sa.1r,169246,Ammodytes,27.4.b ~ 27.4.c,60901,SSB,tonnes,0.515,F,Year-1,NaN,NaN,110000,145000,NaN,NaN,tonnes,NaN,NaN,108944,https://doi.org/10.17895/ices.advice.10000,0,27.4.b
549,2012,Sandeel,san,san.sa.1r,169246,Ammodytes,27.4.b ~ 27.4.c,152970,SSB,tonnes,0.112,F,Year-1,NaN,NaN,110000,145000,NaN,NaN,tonnes,NaN,NaN,45954,https://doi.org/10.17895/ices.advice.10000,1,27.4.c
550,2016,Sandeel,san,san.sa.1r,169246,Ammodytes,27.4.b ~ 27.4.c,231422,SSB,tonnes,0.028,F,Year-1,NaN,NaN,110000,145000,NaN,NaN,tonnes,NaN,NaN,15407,https://doi.org/10.17895/ices.advice.10000,1,27.4.c
551,2020,Sandeel,san,san.sa.1r,169246,Ammodytes,27.4.b ~ 27.4.c,60901,SSB,tonnes,0.515,F,Year-1,NaN,NaN,110000,145000,NaN,NaN,tonnes,NaN,NaN,108944,https://doi.org/10.17895/ices.advice.10000,1,27.4.c


In [23]:
# check what what data is in the custom columns
# stock[~stock.CustomUnits6.isna()].dropna(axis=1)

### OfficialNominalCatches

In [24]:
catches = pd.read_csv(os.path.realpath('data\OfficialNominalCatches\ICESCatchDataset2006-2020.csv')) 

In [25]:
# add country name column
catches['geo'] = catches.Country.map(abbrev_to_country).fillna(catches.Country)

# filter countries of interest
catches = catches[catches.geo.isin(countries)]

# convert Species to lower case
catches.Species = catches.Species.str.lower()

# 2020 has word characters, we divide it into two 
catches[['2020','2020c']] = catches['2020'].str.split(" +",expand = True) 
catches['2020'] = catches['2020'].astype(np.float64)

# keep useful columns
catches = catches.loc[:,['Country','geo','Species','Area','2012','2016','2020']]

In [26]:
# stockMelt.merge(catches, left_on=['speciesAcronym','area'], right_on=['Species', 'Area'])
catches = catches.melt(id_vars=['Country', 'geo','Species', 'Area'], 
       var_name='Year', value_name='CatchesCountry')

In [27]:
catchesPivot = catches.pivot_table( columns='geo', index=['Year','Area', 'Species'] , values='CatchesCountry', aggfunc='sum').reset_index()
catchesPivot['Year'] = catchesPivot['Year'].astype(int)
catchesPivot

geo,Year,Area,Species,Belgium,Denmark,Estonia,Finland,France,Germany,Ireland,Latvia,Lithuania,Netherlands,Poland,Portugal,Spain,Sweden,United Kingdom of GB
0,2012,27,aas,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,27,abk,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.47,0.0,0.0,30.77,NaN,NaN,0.0,NaN
2,2012,27,abx,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2.85,NaN,NaN,NaN
3,2012,27,abz,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,NaN,0.0,NaN,NaN
4,2012,27,acc,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.04,NaN,NaN,1.23,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66214,2020,27_NK,whg,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0
66215,2020,27_NK,wit,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.00,NaN,0.0,0.0
66216,2020,27_NK,wra,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66217,2020,27_NK,wrf,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merge Data

### Merge Stock-Catches

In [61]:
# merge stock and catches data, left or right to check non-matching areas
mStockCatch = stockMelt.merge(catchesPivot, left_on=['speciesAcronym','area','Year'], 
                            right_on=['Species', 'Area', 'Year'], how='outer', indicator=True)
# check the 
checkCatches = mStockCatch[mStockCatch._merge == 'both'].copy()
# mStockCatch[mStockCatch._merge == 'right_only'].Area.unique() # check areas in Catches

checkCatches['official'] = checkCatches[['Belgium', 'Denmark', 'Estonia', 'Finland', 'France',
       'Germany', 'Ireland', 'Latvia', 'Lithuania', 'Netherlands', 'Poland',
       'Portugal', 'Spain', 'Sweden', 'United Kingdom of GB',]].sum(axis=1)

checkCatches.Catches = checkCatches.Catches.astype(np.float64)
checkCatches['diff'] = (checkCatches.official - checkCatches.Catches)/checkCatches.Catches
checkCatches.to_csv('data\checkCatches.csv', index=False)	


In [29]:
print(len(stockMelt), 
len(catchesPivot), 
len(mStockCatch[mStockCatch._merge == 'both']),
len(mStockCatch[mStockCatch._merge == 'left_only']),
len(mStockCatch[mStockCatch._merge == 'right_only']))

3510 66219 2565 945 63750


## Indicators calculation

### TAC/SAD

## Trash

In [30]:
sepAreas = stock["ICES Areas (splited with character '~')"].str.split('~', expand=True)
sepAreas = sepAreas.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [31]:
list1 = stock["ICES Areas (splited with character '~')"].to_list()

newList= []
for item in list1:
    if(str(item) != "nan"):
        newList.append(item)

max_len = -1
for ele in newList: 
    if(len(ele) > max_len): 
        max_len = len(ele) 
        res = ele 
        
print("Longest String is : ", res) 

Longest String is :  27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.10.b ~ 27.12.a.1 ~ 27.12.a.2 ~ 27.12.a.3 ~ 27.12.a.4 ~ 27.12.b ~ 27.12.c ~ 27.14.a ~ 27.14.b.1 ~ 27.14.b.2 ~ 27.2.a.1 ~ 27.2.a.2 ~ 27.2.b.1 ~ 27.2.b.2 ~ 27.3.a ~ 27.3.b.23 ~ 27.3.c.22 ~ 27.3.d.24 ~ 27.3.d.25 ~ 27.3.d.26 ~ 27.3.d.27 ~ 27.3.d.28.1 ~ 27.3.d.28.2 ~ 27.3.d.29 ~ 27.3.d.30 ~ 27.3.d.31 ~ 27.3.d.32 ~ 27.4.a ~ 27.4.b ~ 27.4.c ~ 27.5.a.1 ~ 27.5.a.2 ~ 27.5.b.1.a ~ 27.5.b.1.b ~ 27.5.b.2 ~ 27.6.a ~ 27.6.b.1 ~ 27.6.b.2 ~ 27.7.a ~ 27.7.b ~ 27.7.c.1 ~ 27.7.c.2 ~ 27.7.d ~ 27.7.e ~ 27.7.f ~ 27.7.g ~ 27.7.h ~ 27.7.j.1 ~ 27.7.j.2 ~ 27.7.k.1 ~ 27.7.k.2 ~ 27.8.a ~ 27.8.b ~ 27.8.c ~ 27.8.d.1 ~ 27.8.d.2 ~ 27.8.e.1 ~ 27.8.e.2 ~ 27.9.a ~ 27.9.b.1 ~ 27.9.b.2


In [32]:
pleStock = stock[stock.FishStock == 'san.sa.1r']
pleStock = pleStock["ICES Areas (splited with character '~')"].str.split('~', expand=True).iloc[0,:].to_list()
pleStock = [s.strip() for s in pleStock]

In [33]:
denmarkPle = catches[(catches['Area'].isin(pleStock))  & (catches['Species'] == 'san')].copy()
denmarkPle['2020'] = denmarkPle['2020'].astype(np.float64)
denmarkPle

KeyError: '2020'